In [266]:
import numpy as np
import pandas as p

In [267]:
data = p.read_csv('train.csv', sep=',')

In [268]:
data.head()

,line num,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [269]:
data.shape

(103904, 25)

In [270]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
line num                             103904 non-null int64
id                                   103904 non-null int64
Gender                               103904 non-null object
Customer Type                        103904 non-null object
Age                                  103904 non-null int64
Type of Travel                       103904 non-null object
Class                                103904 non-null object
Flight Distance                      103904 non-null int64
Inflight wifi service                103904 non-null int64
Departure/Arrival time convenient    103904 non-null int64
Ease of Online booking               103904 non-null int64
Gate location                        103904 non-null int64
Food and drink                       103904 non-null int64
Online boarding                      103904 non-null int64
Seat comfort                         103904 non-null int64


Перетираем поля Arrival Delay со значениями null значением из поля Departure Delay

In [271]:
nullrows = data[data['Arrival Delay in Minutes'].isnull()].index
for idx in nullrows:
    data.loc[idx, 'Arrival Delay in Minutes'] = data.loc[idx, 'Departure Delay in Minutes']

In [272]:
data = data.drop(['line num', 'id', 'Arrival Delay in Minutes'], axis=1)

In [273]:
targets = data.groupby('satisfaction').size()/len(data)
print(targets)

satisfaction
neutral or dissatisfied    0.566667
satisfied                  0.433333
dtype: float64


In [274]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [275]:
y = le.fit_transform(data['satisfaction'])
print(list(le.classes_))
print(y)

['neutral or dissatisfied', 'satisfied']
[0 0 1 ... 0 0 0]


In [276]:
data = data.drop(['satisfaction'], axis=1)

In [277]:
X2 = data.select_dtypes(include=[object])
X2.head(3)

,Gender,Customer Type,Type of Travel,Class
0,Male,Loyal Customer,Personal Travel,Eco Plus
1,Male,disloyal Customer,Business travel,Business
2,Female,Loyal Customer,Business travel,Business


In [278]:
columnsToEncode = data.select_dtypes(include=[object]).columns.values

In [279]:
ohe = preprocessing.OneHotEncoder()
dd = ohe.fit_transform(data[columnsToEncode]).toarray()
feature_labels = ohe.categories_
feature_labels = np.concatenate(feature_labels)
print(feature_labels)

['Female' 'Male' 'Loyal Customer' 'disloyal Customer' 'Business travel'
 'Personal Travel' 'Business' 'Eco' 'Eco Plus']


In [280]:
data[feature_labels] = p.DataFrame(dd)

In [281]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Departure Delay in Minutes,Female,Male,Loyal Customer,disloyal Customer,Business travel,Personal Travel,Business,Eco,Eco Plus
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,25,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,11,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [282]:
data = data.drop(columnsToEncode, axis=1)

In [283]:
data.head()

,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,...,Departure Delay in Minutes,Female,Male,Loyal Customer,disloyal Customer,Business travel,Personal Travel,Business,Eco,Eco Plus
0,13,460,3,4,3,1,5,3,5,5,...,25,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,25,235,3,2,3,3,1,3,1,1,...,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,26,1142,2,2,2,2,5,5,5,5,...,0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,25,562,2,5,5,5,2,2,2,2,...,11,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,61,214,3,3,3,3,4,5,5,3,...,0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [284]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.30, random_state=5)

In [285]:
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [294]:
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [295]:
clfrs = [Perceptron(max_iter=1000, tol=1e-3), LinearSVC(random_state=0, tol=1e-5, max_iter=2000),
         DecisionTreeClassifier(max_depth=10), LogisticRegression(multi_class='auto',solver='liblinear')]
for clfr in clfrs:
    clfr.fit(X_train_scaled, y_train)
    pred_test = clfr.predict(X_test_scaled)
    print('{}:{:.2%}\n'.format(clfr.__class__.__name__, metrics.accuracy_score(y_test, pred_test)))

Perceptron:82.52%



D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC:87.16%

DecisionTreeClassifier:94.33%

LogisticRegression:87.25%



In [291]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [292]:
y_pred = rfc.predict(X_test)

In [293]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.9624663159245477